In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pylab inline

In [ ]:
mainpath='../csse_covid_19_data/csse_covid_19_time_series/'

In [ ]:
def getCountryStats(country='world',printinfo=True,plot=True,savefig=False):
    data=dict()
    cases=['confirmed','recovered','deaths']
    for case in cases:            
        df=pd.read_csv(mainpath+'time_series_covid19_{}_global.csv'.format(case))
        days=df.columns[4:]
        
        if country.lower()!='world':
            isCountry=df['Country/Region']==country
            df=df[isCountry]
        
        totalcases= np.array(df[days].sum())
        
        if case=='confirmed':
            start=totalcases>0
            # Days after first confirmed
            data['days']= days[start]
        data[case]=totalcases[start]  
        
    if printinfo:  
        print('COVID-19 FIRST DAY IN {}: {}'.format(country.upper(),data['days'][0])) 
        print('  TOTAL CASES IN {}'.format(country.upper()))
        print('=========================')
        for case in cases:
            print('{:9s}: {:6d}'.format(case.upper(),int(max(data[case]))))
        print('')
    
    if plot:
        plt.figure(figsize=(16,6))       
        for case in cases:
            plt.plot_date(data['days'],data[case],'-o',label=case)
        plt.title('{} COVID-19 CASES'.format(country.upper()))
        plt.ylabel('Cases')
        plt.xlabel('Days')
        plt.legend()
        plt.tight_layout()
        plt.grid(linestyle='--')
        if max(data['confirmed'])>50000:
            plt.yscale('log')
        if len(data['days'])>30:
            plt.xticks(data['days'][::10])
        plt.tick_params(axis='x',rotation=90)
        if savefig:
            plt.savefig('Plots/{}.png'.format(country.lower()),bbox_inches = "tight")
    return data

def plot_multicountry(countries,case='confirmed',savefig=False):
    plt.figure(figsize=(16,6))
    plt.title('{}'.format(case.upper()))
    plt.ylabel('Cases')
    plt.xlabel('Days after First Confirmed')
    for country in countries:
        countrystats=getCountryStats(country,printinfo=False,plot=False)
        days=np.arange(len(countrystats['days']))
        plt.plot(days,countrystats[case.lower()],'-o',label=country)
    plt.grid(linestyle='--')
    plt.legend()
    plt.tight_layout()
    if savefig:
        plt.savefig('Plots/multiple_{}.png'.format(case.lower()))
        
def plot_dailycases(countries,savefig=False):
    plt.figure(figsize=(16,6))
    plt.title('Daily Cases')
    plt.ylabel('Cases')
    plt.xlabel('Days after First Confirmed')
    for country in countries:
        countrystats=getCountryStats(country,printinfo=False,plot=False)
        daily=np.zeros(len(countrystats['confirmed']))
        daily[1:] = countrystats['confirmed'][1:]-countrystats['confirmed'][:-1]
        plt.bar(countrystats['days'],daily,label=country)
    plt.grid(linestyle='--')
    plt.legend()
    plt.tick_params(axis='x',rotation=90)
    plt.tight_layout()
    if savefig:
        plt.savefig('Plots/daily.png')

def plot_active(countries,savefig=False):
    plt.figure(figsize=(16,6))
    plt.ylabel('Active Cases')
    plt.xlabel('Days after First Confirmed')
    for country in countries:
        countrystats=getCountryStats(country,printinfo=False,plot=False)
        active=countrystats['confirmed']-countrystats['recovered']-countrystats['deaths']
        days=np.arange(len(countrystats['days']))
        plt.bar(countrystats['days'],active,label=country)
    plt.grid(linestyle='--')
    plt.legend()
    plt.tight_layout()
    plt.tick_params(axis='x',rotation=90)
    if savefig:
        plt.savefig('Plots/multiple_{}.png'.format(case.lower()))

# World

In [ ]:
stats=getCountryStats(savefig=True)

# Mexico

In [ ]:
mxStats=getCountryStats('Mexico',savefig=True)

# Latin America

In [ ]:
countries=['Argentina','Mexico','Colombia','Chile','Brazil','Ecuador','Peru']
plot_multicountry(countries,case='confirmed',savefig=True)
plot_multicountry(countries,case='recovered',savefig=True)
plot_multicountry(countries,case='deaths',savefig=True)

In [ ]:
plot_dailycases(['Mexico'],savefig=False)

In [ ]:
plot_active(['Mexico'],savefig=False)

# Monte-Carlo

In [ ]:
Stats = getCountryStats('Mexico',plot=False,savefig=True)
days = np.arange(len(Stats['days']))
count = Stats['confirmed']

In [ ]:
max(count[1:]-count[:-1])

In [ ]:
import emcee

In [ ]:
from scipy.optimize import minimize
from scipy.special import factorial

def model(params,days):
        #mod=params[0]*np.exp(-0.5*(days-params[1])**2/params[2]**2) #Gaussian
        #mod = params[0]*np.exp(params[1]*(days-params[2]))/(1+np.exp(params[1]*(days-params[2])))**2.#Logistic derivative
        #mod = params[0]*params[1]**days*np.exp(-params[1])/factorial(days)
        #mod = params[0]*days**3/(np.exp(params[1]*days)-1) + params[2] # Plank?
        mod = params[0]/(1+params[1]*np.exp(-params[2]*days))# Logistic model
        return mod
    
def log_like(params,days,count):
    chi = (count-model(params,days))**2.
    return -0.5*sum(chi)

def priors(params,plist):
    p=True
    for i,param in enumerate(params):
        p &= plist[2*i]<param<plist[2*i+1]
    if p==True:
        return 0.0
    return -np.inf

def log_post(params,days,count,plist):
    prior = priors(params,plist)
    if not np.isfinite(prior):
        return -np.inf
    return prior + log_like(params,days,count)

In [ ]:
nwalkers = 300
ndim = 3

In [ ]:
from scipy.optimize import minimize
fun = lambda *args: -log_like(*args)
results = minimize(fun,[max(count),3000,1],args=(days,count),method='L-BFGS-B')
results

In [ ]:
mid = results.x
plist=[10000,3e6,0,1e6,0,10]
pos = mid + 1e-4 * np.random.randn(nwalkers, ndim)

In [ ]:
b_steps, steps = 400,1500

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, ndim,log_post, args=(days,count,plist))
pos,prob,state=sampler.run_mcmc(pos, b_steps)
chain_burn  = sampler.get_chain()

In [ ]:
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
labels = [r'$L$',r'$\beta$',r'$k$']
for i in range(ndim):
    ax = axes[i]
    ax.plot(chain_burn[:, :, i],'k', alpha=0.3)
    ax.set_xlim(0, len(chain_burn))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.grid(linestyle='--')
axes[-1].set_xlabel("step number")
plt.suptitle("Burn In")

In [ ]:
sampler.acceptance_fraction.mean()

In [ ]:
sampler.reset()
_,_,_=sampler.run_mcmc(pos, steps, rstate0=state)
sampler.acceptance_fraction.mean()

In [ ]:
chain  = sampler.get_chain()
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
labels = [r'$L$',r'$\beta$',r'$k$']
for i in range(ndim):
    ax = axes[i]
    ax.plot(chain[:,:, i],'k', alpha=0.3)
    ax.set_xlim(0, len(chain))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)
    ax.grid(linestyle='--')

axes[-1].set_xlabel("step number");

In [ ]:
import corner as corner

In [ ]:
labels = [r'$L$',r'$\beta$',r'$k$']
fig = corner.corner(sampler.flatchain,labels=labels,bins=50,plot_datapoints=False,smooth=2.,smooth1d=2.,
                    levels=(1.-np.exp(-0.5),1.-np.exp(-2.),1.-np.exp(-4.5)),quantiles=[.025,0.5,0.975],show_titles=True);

In [ ]:
chain.mean(axis=(0,1))

In [ ]:
figure = plt.figure(figsize=(16,6))
d_pred=np.arange(300)
runs=sampler.flatchain[np.random.randint(len(sampler.flatchain), size=300)]
for c1,c2,c3 in runs:
    fig1,=plt.plot(d_pred,model([c1,c2,c3],d_pred),'g',alpha=0.3)
fig2, = plt.plot(d_pred,model(chain.mean(axis=(0,1)),d_pred),'r')
fig3 = plt.bar(days,count,color='b',width=1,edgecolor='k',alpha=0.5)

plt.legend([fig1,fig2,fig3],['Random samples','mean','data'],loc='best')
plt.xlabel('days',fontsize=20)
plt.ylabel(r'Confirmed Cases',fontsize=20)
plt.grid(linestyle='--')

In [ ]:
figure = plt.figure(figsize=(16,6))
d_pred=np.arange(200)
for c1,c2,c3 in runs:
    mod = model([c1,c2,c3],d_pred)
    pred = np.zeros(len(d_pred))
    pred[1:] = mod[1:]-mod[:-1]
    fig1,=plt.plot(d_pred,pred,'g',alpha=0.3);
mod=model(chain.mean(axis=(0,1)),d_pred)
pred[1:] = mod[1:]-mod[:-1]
fig2, = plt.plot(d_pred,pred,'r');

real = np.zeros(len(days))
real[1:] = count[1:]-count[:-1]
fig3 = plt.bar(days,real,color='b',width=1,edgecolor='k',alpha=0.5);
plt.legend([fig1,fig2,fig3],['Random samples','mean','data'],loc='best')
plt.xlabel('days',fontsize=20)
plt.ylabel(r'Cases by day',fontsize=20)
plt.grid(linestyle='--')

In [ ]:
maximum=np.argmax(pred)
day=np.where((d_pred>40)&(pred<0.02*pred[maximum]))[0]

In [ ]:
from datetime import datetime,timedelta 
start_date=datetime.strptime(Stats['days'][0],"%m/%d/%y")
end_date= start_date+timedelta(days=int(day[0]))
print(f'End of quarantine: {end_date.strftime("%d/%m/%Y")}')

In [ ]:
max_date= start_date+timedelta(days=int(maximum))
print(f'Maximum expected: {max_date.strftime("%d/%m/%Y")}')

In [ ]:
count[1:]-count[:-1]